# Trajectories of role + trait space

In [2]:
import json
import os
import sys
import torch
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.subplots as sp
import torch.nn.functional as F
import re

sys.path.append('.')
sys.path.append('..')

from utils.inference_utils import *
from utils.probing_utils import *


INFO 09-18 16:36:45 [__init__.py:216] Automatically detected platform cuda.


In [3]:
# load in PCA results
CHAT_MODEL_NAME = "Qwen/Qwen3-32B"
model_readable = "Qwen 3 32B"
model_short = model_readable.replace(" ", "-").lower()
layer = 32


In [4]:
# filename = "auto40"
components = 6

In [5]:
role_dir = "roles_240" 
trait_dir = "traits_240"

acts_input_dir = f"/workspace/{model_short}/dynamics"
plot_output_dir = f"/root/git/plots/{model_short}/trajectory"
os.makedirs(plot_output_dir, exist_ok=True)

## Load and project all conversation activations 

In [6]:
# load in activations
role_results = torch.load(f"/workspace/{model_short}/{role_dir}/pca/layer{layer}_pos23.pt", weights_only=False)
trait_results = torch.load(f"/workspace/{model_short}/{trait_dir}/pca/layer{layer}_pos-neg50.pt", weights_only=False)


In [7]:
def pc_cosine_similarity(mean_acts_per_turn, pca_results, n_pcs=8):
    if isinstance(mean_acts_per_turn, list):
        stacked_acts = torch.stack(mean_acts_per_turn)
    else:
        stacked_acts = mean_acts_per_turn
    normalized_acts = F.normalize(stacked_acts, dim=1)
    normalized_pcs = pca_results['pca'].components_[:n_pcs] / np.linalg.norm(pca_results['pca'].components_[:n_pcs], axis=1, keepdims=True)
    cosine_sims = normalized_acts.float().numpy() @ normalized_pcs.T
    return cosine_sims

def pc_projection(mean_acts_per_turn, pca_results, n_pcs=8):
    if isinstance(mean_acts_per_turn, list):
        stacked_acts = torch.stack(mean_acts_per_turn)
    else:
        stacked_acts = mean_acts_per_turn
    stacked_acts = stacked_acts.float().numpy()
    scaled_acts = pca_results['scaler'].transform(stacked_acts)
    projected_acts = pca_results['pca'].transform(scaled_acts)
    return projected_acts[:, :n_pcs]
    

    

In [7]:
# load every activation file in the directory if the name matches a regex
all_objs = []
# for file in os.listdir(acts_input_dir):
#     if file.endswith('.pt'):
#         obj = torch.load(f"{acts_input_dir}/{file}", weights_only=False, map_location="cpu")
#         A = obj['activations'][:, layer, :]
#         A = A[::2]

#         role_sims = pc_cosine_similarity(A, role_results, r_components)
#         trait_sims = pc_cosine_similarity(A, trait_results, t_components)
#         print(role_sims.shape)

#         role_projs = pc_projection(A, role_results, r_components)
#         trait_projs = pc_projection(A, trait_results, t_components)
#         print(role_projs.shape)

#         obj['role_sims'] = role_sims
#         obj['trait_sims'] = trait_sims
#         obj['role_projs'] = role_projs
#         obj['trait_projs'] = trait_projs

#         all_objs.append(obj)

#         torch.save(obj, f"{acts_input_dir}/projected/{file}")
for file in os.listdir(f"{acts_input_dir}/projected"):

    obj = torch.load(f"{acts_input_dir}/projected/{file}", weights_only=False, map_location="cpu")
    all_objs.append(obj)



In [ ]:
all_role_projs = [obj['role_projs'] for obj in all_objs]
all_trait_projs = [obj['trait_projs'] for obj in all_objs]


In [18]:
sliced_role_projs = [obj[:, :components] for obj in all_role_projs]
sliced_trait_projs = [obj[:, :components] for obj in all_trait_projs]


In [24]:
files = [f"{obj['domain']}_persona{obj['persona_id']}_topic{obj['topic_id']}" for obj in all_objs]
print(files[:10])

['writing_persona9_topic9', 'writing_persona9_topic8', 'writing_persona9_topic7', 'writing_persona9_topic6', 'writing_persona9_topic5', 'writing_persona9_topic4', 'writing_persona9_topic3', 'writing_persona9_topic2', 'writing_persona9_topic19', 'writing_persona9_topic18']


In [23]:
print(obj.keys())

dict_keys(['model', 'auditor_model', 'domain', 'persona_id', 'persona', 'topic_id', 'topic', 'turns', 'conversation', 'activations', 'role_sims', 'trait_sims', 'role_projs', 'trait_projs'])


In [25]:
df = pd.DataFrame()
df['filenames'] = files


In [ ]:
# calculate the cumulative path length for each conversation
role_path_lengths = [np.sum(np.linalg.norm(np.diff(obj['role_projs'], axis=0), axis=1)) for obj in all_objs]
trait_path_lengths = [np.sum(np.linalg.norm(np.diff(obj['trait_projs'], axis=0), axis=1)) for obj in all_objs]

df['role_path_length'] = role_path_lengths
df['trait_path_length'] = trait_path_lengths

In [9]:
# load csv from file into df
df = pd.read_csv(f"./path_lengths.csv")

In [12]:
# also get largest single jump
role_max_jump = [np.max(np.linalg.norm(np.diff(obj['role_projs'], axis=0), axis=1)) for obj in all_objs]
trait_max_jump = [np.max(np.linalg.norm(np.diff(obj['trait_projs'], axis=0), axis=1)) for obj in all_objs]
df['role_max_jump'] = role_max_jump
df['trait_max_jump'] = trait_max_jump

df.to_csv(f"./path_lengths.csv", index=False)


In [13]:
# role pc1 biggest jump and distance
role_pc1_path_lengths = [
    np.sum(np.abs(np.diff(obj['role_projs'][:, 0])))  # Only PC1 (column 0)
    for obj in all_objs
]

role_pc1_max_jumps = [
    np.max(np.abs(np.diff(obj['role_projs'][:, 0])))  # Max jump in PC1
    for obj in all_objs
]

df['role_pc1_path_length'] = role_pc1_path_lengths
df['role_pc1_max_jump'] = role_pc1_max_jumps


df.to_csv(f"./path_lengths.csv", index=False)

## Cosine similarity with trait and role PCs

## Plot trajectory

In [ ]:
def plot_mean_response_trajectory(similarity_matrix, conversation=None, title=None, pc_titles=None, projection=False):
    """
    Create a single line plot showing mean response per turn.
    
    Parameters:
    - similarity_matrix: Numpy matrix of shape (n_turns, n_pcs)
    - conversation: Optional conversation data for turn context
    - title: Optional custom title
    - pc_titles: Optional list of PC titles
    
    Returns:
    - Plotly figure object
    """
    
    print("Creating mean response trajectory plot...")
    
    # Get dimensions
    n_turns, n_pcs = similarity_matrix.shape
    turn_indices = np.arange(n_turns)
    
    # Create default PC titles if not provided
    if pc_titles is None:
        pc_titles = [f"PC{i+1}" for i in range(n_pcs)]
    
    # Define color palette for PCs
    pc_colors = [
        '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', 
        '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
        '#bcbd22', '#17becf', '#aec7e8', '#ffbb78'
    ]
    
    # Helper function to wrap text for hover display
    def wrap_text(text, max_chars_per_line=70):
        """Wrap text to specified line length with HTML breaks."""
        if len(text) <= max_chars_per_line:
            return text
        
        words = text.split()
        lines = []
        current_line = []
        current_length = 0
        
        for word in words:
            if current_length + len(word) + len(current_line) > max_chars_per_line:
                if current_line:  # Don't add empty lines
                    lines.append(' '.join(current_line))
                current_line = [word]
                current_length = len(word)
            else:
                current_line.append(word)
                current_length += len(word)
        
        if current_line:  # Add the last line
            lines.append(' '.join(current_line))
        
        return '<br>'.join(lines)
    
    # Create enhanced turn context for hover text
    turn_contexts = []
    if conversation is not None:
        assistant_turns = [i for i, turn in enumerate(conversation) if turn['role'] == 'assistant']
        for turn_idx in range(n_turns):
            if turn_idx < len(assistant_turns):
                conv_turn_idx = assistant_turns[turn_idx]
                if conv_turn_idx < len(conversation):
                    # Get assistant response
                    assistant_content = conversation[conv_turn_idx]['content']
                    
                    # Get preceding user question (if exists)
                    user_content = ""
                    if conv_turn_idx > 0 and conversation[conv_turn_idx - 1]['role'] == 'user':
                        user_content = conversation[conv_turn_idx - 1]['content']
                    
                    # Format hover text with both user question and assistant response
                    hover_parts = [f"<b>Turn {turn_idx}</b>"]
                    
                    if user_content:
                        # Truncate user content to reasonable length and wrap
                        user_truncated = user_content[:200] + "..." if len(user_content) > 200 else user_content
                        user_wrapped = wrap_text(user_truncated, 70)
                        hover_parts.append(f"<b>User:</b> {user_wrapped}")
                    
                    # Show more of the assistant response (150-200 chars) and wrap
                    assistant_truncated = assistant_content[:300] + "..." if len(assistant_content) > 180 else assistant_content
                    assistant_wrapped = wrap_text(assistant_truncated, 70)
                    hover_parts.append(f"<b>Assistant:</b> {assistant_wrapped}")
                    
                    turn_contexts.append('<br>'.join(hover_parts))
                else:
                    turn_contexts.append(f"<b>Turn {turn_idx}</b>")
            else:
                turn_contexts.append(f"<b>Turn {turn_idx}</b>")
    else:
        turn_contexts = [f"<b>Turn {turn_idx}</b>" for turn_idx in range(n_turns)]
    
    # Create Plotly figure
    fig = go.Figure()
    
    # Add line traces for each PC with markers
    for pc_idx in range(n_pcs):
        pc_name = pc_titles[pc_idx]
        similarities = similarity_matrix[:, pc_idx]
        color = pc_colors[pc_idx % len(pc_colors)]
        
        fig.add_trace(go.Scatter(
            x=turn_indices,
            y=similarities,
            mode='lines+markers',
            name=pc_name,
            line=dict(color=color, width=2),
            marker=dict(color=color, size=4, opacity=0.8),
            hovertemplate='<b>%{fullData.name}</b><br>' +
                         '%{text}<br>' +
                         '<b>Cosine Similarity:</b> %{y:.3f}<br>' +
                         '<extra></extra>',
            text=turn_contexts
        ))
    
    # Update layout
    default_title = f"Mean Response PC Trajectory"
    if projection:
        yaxis_title = "PC Projection"
    else:
        yaxis_title = "Cosine Similarity with PC"
    
    fig.update_layout(
        title=dict(
            text=title if title else default_title,
            x=0.5,
            font=dict(size=16),
            subtitle={"text": f"{model_readable}, Layer {layer}"}
        ),
        xaxis_title="Conversation Turn",
        yaxis_title=yaxis_title,
        width=1400,
        height=600,
        hovermode='closest',
        legend=dict(
            yanchor="middle",
            y=0.5,
            xanchor="left",
            x=1.02,
            bgcolor="rgba(255,255,255,0.8)"
        )
    )
    
    # Add grid for easier reading
    fig.update_xaxes(
        showgrid=True, 
        gridwidth=1, 
        gridcolor='lightgray',
        zeroline=True,
        tick0=0,
        dtick=1  # Show every turn
    )
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', zeroline=True)
    
    # Add light vertical lines between turns for clarity
    for turn_idx in range(1, n_turns):
        fig.add_vline(
            x=turn_idx - 0.5,
            line_dash="dot",
            line_color="lightgray",
            line_width=1,
            opacity=0.3
        )
    
    print(f"Created trajectory plot with {n_pcs} PC lines across {n_turns} turns")
    
    return fig

In [ ]:
# Create role PC titles
if model_short == "qwen-3-32b":
    role_pc_titles = ['Assistant-like ↔ role-playing', "mystical/transcendent ↔ mundane/irreverent", "empathetic/vulnerable ↔ analytical/predatory", "concrete/practical ↔ abstract/ideological", "thinking/passive ↔ doing/active", "creative/expressive ↔ rigid/constrained"]
elif model_short == "gemma-2-27b":
    role_pc_titles = ['Assistant-like ↔ role-playing', "inhuman ↔ human", "independent ↔ dependent", "nurturing ↔ adversarial", "social ↔ technical", "structured ↔ liminal"]

# # Plot role trajectory
# role_fig = plot_mean_response_trajectory(
#     role_sims, 
#     conversation=conversation, 
#     title=f"Conversation Trajectory in Role PC Space: {filename.capitalize()}", 
#     pc_titles=role_pc_titles
# )
# role_fig.show()
# role_fig.write_html(f"{plot_output_dir}/{filename}_role_cossim.html")

# role_fig = plot_mean_response_trajectory(
#     role_projs, 
#     conversation=conversation, 
#     title=f"Conversation Trajectory in Role PC Space: {filename.capitalize()}", 
#     pc_titles=role_pc_titles,
#     projection=True
# )
# role_fig.show()
# role_fig.write_html(f"{plot_output_dir}/{filename}_role_proj.html")

In [ ]:
# Create trait PC titles
if model_short == "qwen-3-32b":
    trait_pc_titles = ["expressive/irreverent ↔ controlled/professional", "analytical ↔ intuitive", "accessible/practical ↔ esoteric/complex", "active/flexible ↔ passsive/rigid", "questioning ↔ confident", "indirect/diplomatic ↔ direct/assertive"]
elif model_short == "gemma-2-27b":
    trait_pc_titles = ["benevolent ↔ evil", "analytical ↔ emotional", "accommodating ↔ confrontational", "grounded ↔ mystical", "thoughtful ↔ decisive", "conformist ↔ defiant"]

# Plot trait trajectory
# trait_fig = plot_mean_response_trajectory(
#     trait_sims, 
#     conversation=conversation, 
#     title=f"Conversation Trajectory in Trait PC Space: {filename.capitalize()}", 
#     pc_titles=trait_pc_titles
# )
# trait_fig.show()
# trait_fig.write_html(f"{plot_output_dir}/{filename}_trait_cossim.html")

# trait_fig = plot_mean_response_trajectory(
#     trait_projs, 
#     conversation=conversation, 
#     title=f"Conversation Trajectory in Trait PC Space: {filename.capitalize()}", 
#     pc_titles=trait_pc_titles,
#     projection=True
# )

# trait_fig.show()
# trait_fig.write_html(f"{plot_output_dir}/{filename}_trait_proj.html")

In [ ]:
# Create a combined plot showing both role and trait trajectories

import plotly.subplots as sp


# Create subplot figure
combined_fig = sp.make_subplots(
    rows=2, cols=1,
    subplot_titles=("Trajectory in Role PC Space", "Trajectory in Trait PC Space"),
    vertical_spacing=0.1
)

for tr in plot_mean_response_trajectory(
        role_projs, conversation=None, title="<b>Role PCs</b>", pc_titles=role_pc_titles, projection=True
    ).data:
    tr.update(
        showlegend=True,
        legendgroup="role",
        legendgrouptitle_text="Role PCs"
    )
    combined_fig.add_trace(tr, row=1, col=1)

# Add trait traces to second subplot
for tr in plot_mean_response_trajectory(
        trait_projs, conversation=None, title="<b>Trait PCs</b>", pc_titles=trait_pc_titles, projection=True
    ).data:
    tr.update(
        showlegend=True,
        legendgroup="trait",
        legendgrouptitle_text="Trait PCs"
    )
    combined_fig.add_trace(tr, row=2, col=1)

combined_fig.update_layout(
    height=800,
    width=1200,
    title={
        "text": f"Conversation Trajectories in Persona Subspace: 100 Coding Conversations (code blocks filtered out)",
        "subtitle": {"text": f"{model_readable}, Layer {layer}"}
    },
    showlegend=True,
    legend=dict(
        traceorder="grouped",     # keeps groups together
        groupclick="toggleitem",
        x=1.02, xanchor="left",   # place legend outside on the right
        y=1, yanchor="top"
    ),
    legend_tracegroupgap=12       # space between groups
)
# Bottom x-axis label only
combined_fig.update_xaxes(title_text="Conversation Turn", row=2, col=1)

# Y-axis labels for both subplots
combined_fig.update_yaxes(title_text="PC Projection", row=1, col=1)
combined_fig.update_yaxes(title_text="PC Projection", row=2, col=1)

combined_fig.show()
combined_fig.write_html(f"{plot_output_dir}/auto_coding_no_code.html")